In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from scipy.stats import entropy
import seaborn as sns

In [ ]:
directory = "keywordTCGA/brca/"
os.chdir(directory)

In [ ]:
df_first = pd.read_csv("topsbm/topsbm_level_2_topic-dist.csv", index_col=1).drop("i_doc", 1)
df_second = pd.read_csv("trisbm/trisbm_level_1_metadatum-dist.csv", index_col=1).drop("i_doc", 1)
df_second = df_second.reindex(index=df_first.index)

In [ ]:
tf.shape([1,2,3])

In [ ]:
@tf.function
def get_entropy(x: tf.Tensor, y: tf.Tensor, epsilon = 1e-7)->float:
    #entropy(x.numpy(), y.numpy(), base=np.exp(1))
    #tf.keras.losses.KLD(x,y)
    
    x = tf.clip_by_value(x, tf.constant(epsilon, dtype=x.dtype), tf.constant(1, dtype=x.dtype))
    y = tf.clip_by_value(y, tf.constant(epsilon, dtype=y.dtype), tf.constant(1, dtype=y.dtype))
    
    return x*tf.math.log(x/y)

@tf.function
def get_array_entropy(first: tf.Tensor, second: tf.Tensor):
    return tf.map_fn(lambda xi: 
          tf.map_fn(lambda yi: get_entropy(xi,yi), second),
          first)

@tf.function
def get_all_rows_entropy(first: tf.Tensor, second: tf.Tensor):
    return tf.reduce_sum(
        tf.map_fn(lambda xi: 
          tf.reduce_sum(
              tf.map_fn(lambda yi: get_array_entropy(xi,yi), second),
              axis=0),
          first, parallel_iterations=12),
        axis=0)

In [ ]:
first = tf.convert_to_tensor(df_first.values, dtype=tf.float64)
second = tf.convert_to_tensor(df_second.values, dtype=tf.float64)
get_array_entropy(first[0], second[1])[:2,:3]

In [ ]:
kld_matrix = get_array_entropy(tf.reduce_mean(first,0), np.nanmean(second, 0)).numpy()

In [ ]:
sns.heatmap(kld_matrix/kld_matrix.max(),
           xticklabels=df_second.columns,
           yticklabels=df_first.columns
           )

In [ ]:
kld_matrix